In [1]:
!pip install wandb

5c10d803d459c2f6e00069378b89b7450961222eRequirement already satisfied: wandb in /usr/local/lib/python3.10/dist-packages (0.17.5)


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import TrainerCallback, TrainingArguments
from sklearn.model_selection import train_test_split
import wandb

# Initialize W&B project
wandb.init(project="ner-with-bert-sapient2")

# Load the data
data_path = 'ner_dataset.csv'  # Adjust this path as necessary
df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Check the data
print(df.head())
# 5c10d803d459c2f6e00069378b89b7450961222e

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


    Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O


In [3]:
# Fill NaN values
df = df.fillna(method='ffill')

# Group data by sentences
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = df.groupby("Sentence #").apply(agg_func)

sentences = [[word for word, _, _ in sent] for sent in grouped]
labels = [[tag for _, _, tag in sent] for sent in grouped]

# Initialize BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Tokenize sentences
tokenized_inputs = tokenizer(sentences, is_split_into_words=True, truncation=True, padding=True)

# Convert labels to integers
label_list = list(set(df['Tag'].values))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

# Align labels with tokens
def align_labels(tokenized_inputs, labels):
    aligned_labels = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_id = None
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_word_id:
                label_ids.append(label_to_id[label[word_id]])
            else:
                label_ids.append(-100)
            previous_word_id = word_id
        aligned_labels.append(label_ids)
    return aligned_labels

aligned_labels = align_labels(tokenized_inputs, labels)


<ipython-input-3-16f40d4771e4>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset
dataset = NERDataset(tokenized_inputs, aligned_labels)


In [5]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [6]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Create a custom W&B callback
class WandbCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            wandb.log(logs)

In [8]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to="wandb"
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)


In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.708400
20,2.620100
30,2.416700
40,2.109800
50,1.652200
60,1.134300
70,0.781600
80,0.640100
90,0.616900
100,0.499500


TrainOutput(global_step=2799, training_loss=0.14195805140672815, metrics={'train_runtime': 813.9091, 'train_samples_per_second': 54.968, 'train_steps_per_second': 3.439, 'total_flos': 1849669954876242.0, 'train_loss': 0.14195805140672815, 'epoch': 3.0})

In [ ]:
# model.save_pretrained("/ner_model")
# tokenizer.save_pretrained("./ner_model")

('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

In [10]:
# Save the model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_token")

('./ner_token/tokenizer_config.json',
 './ner_token/special_tokens_map.json',
 './ner_token/vocab.txt',
 './ner_token/added_tokens.json',
 './ner_token/tokenizer.json')

In [11]:
# Load model and tokenizer
model = BertForTokenClassification.from_pretrained("./ner_model")
tokenizer = BertTokenizerFast.from_pretrained("./ner_token")

# Example sentence
sentence = "Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country"

# Tokenize input
inputs = tokenizer(sentence, return_tensors="pt")

# Get predictions
outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

# Convert predictions to labels
predicted_labels = [id_to_label[label_id] for label_id in predictions[0].tolist()]
print(predicted_labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(id_to_label)
print(label)

{0: 'B-org', 1: 'I-eve', 2: 'B-per', 3: 'I-org', 4: 'B-eve', 5: 'B-nat', 6: 'I-gpe', 7: 'B-art', 8: 'I-tim', 9: 'I-per', 10: 'O', 11: 'I-geo', 12: 'I-nat', 13: 'I-art', 14: 'B-tim', 15: 'B-geo', 16: 'B-gpe'}
{'B-org': 0, 'I-eve': 1, 'B-per': 2, 'I-org': 3, 'B-eve': 4, 'B-nat': 5, 'I-gpe': 6, 'B-art': 7, 'I-tim': 8, 'I-per': 9, 'O': 10, 'I-geo': 11, 'I-nat': 12, 'I-art': 13, 'B-tim': 14, 'B-geo': 15, 'B-gpe': 16}


In [12]:
import shutil
import os

# Directory to be zipped
directory_to_zip = './ner_model'

# Name of the resulting zip file
zip_filename = 'ner_model.zip'

# Create the zip file
shutil.make_archive('ner_model', 'zip', directory_to_zip)

# Ensure the zip file is in the current directory
zip_filepath = 'ner_model.zip'
if os.path.exists(zip_filepath):
    print(f"{zip_filepath} created successfully!")

    # If running in a Jupyter or Colab environment, use the following line to download the file
    from google.colab import files
    files.download(zip_filepath)
else:
    print(f"Failed to create {zip_filepath}!")


ner_model.zip created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import shutil
import os

# Directory to be zipped
directory_to_zip = './ner_token'

# Name of the resulting zip file
zip_filename = 'ner_token.zip'

# Create the zip file
shutil.make_archive('ner_token', 'zip', directory_to_zip)

# Ensure the zip file is in the current directory
zip_filepath = 'ner_token.zip'
if os.path.exists(zip_filepath):
    print(f"{zip_filepath} created successfully!")

    # If running in a Jupyter or Colab environment, use the following line to download the file
    from google.colab import files
    files.download(zip_filepath)
else:
    print(f"Failed to create {zip_filepath}!")


ner_token.zip created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
import os

# Directory to be zipped
directory_to_zip = './wandb'

# Name of the resulting zip file
zip_filename = 'wandb.zip'

# Create the zip file
shutil.make_archive('wandb', 'zip', directory_to_zip)

# Ensure the zip file is in the current directory
zip_filepath = 'wandb.zip'
if os.path.exists(zip_filepath):
    print(f"{zip_filepath} created successfully!")

    # If running in a Jupyter or Colab environment, use the following line to download the file
    from google.colab import files
    files.download(zip_filepath)
else:
    print(f"Failed to create {zip_filepath}!")


wandb.zip created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
import os

# Directory to be zipped
directory_to_zip = './results'

# Name of the resulting zip file
zip_filename = 'results.zip'

# Create the zip file
shutil.make_archive('results', 'zip', directory_to_zip)

# Ensure the zip file is in the current directory
zip_filepath = 'results.zip'
if os.path.exists(zip_filepath):
    print(f"{zip_filepath} created successfully!")

    # If running in a Jupyter or Colab environment, use the following line to download the file
    from google.colab import files
    files.download(zip_filepath)
else:
    print(f"Failed to create {zip_filepath}!")


results.zip created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
model.save_pretrained("/content/gdrive/My Drive/ner_model")
tokenizer.save_pretrained("/content/gdrive/My Drive/ner_token")

('/content/gdrive/My Drive/ner_token/tokenizer_config.json',
 '/content/gdrive/My Drive/ner_token/special_tokens_map.json',
 '/content/gdrive/My Drive/ner_token/vocab.txt',
 '/content/gdrive/My Drive/ner_token/added_tokens.json',
 '/content/gdrive/My Drive/ner_token/tokenizer.json')

In [1]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments

In [2]:
# Load model and tokenizer
model = BertForTokenClassification.from_pretrained("/content/gdrive/My Drive/ner_model")
tokenizer = BertTokenizerFast.from_pretrained("/content/gdrive/My Drive/ner_token")

# Example sentence
sentence = "Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country"

# Tokenize input
inputs = tokenizer(sentence, return_tensors="pt")

# Get predictions
outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

# Convert predictions to labels
predicted_labels = [id_to_label[label_id] for label_id in predictions[0].tolist()]
print(predicted_labels)

NameError: name 'BertForTokenClassification' is not defined